In [2]:
import string
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mahha\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [3]:
def nettoyage(corpus_ensemble_documents):
    for i in range(len(corpus_ensemble_documents)):
        corpus_ensemble_documents[i]=corpus_ensemble_documents[i].lower()
    for i in range(len(corpus_ensemble_documents)):
        for c in string.ponctuation:
            x=corpus_ensemble_documents[i].replace(c, ' ')
            corpus_ensemble_documents[i]=x
    stopwords_anglais=stopwords.words('english')
    for i in range(len(corpus_ensemble_documents)):
        L=corpus_ensemble_documents[i].split()
        for mot in L:
            if mot in stopwords_anglais:
                while mot in L:
                    L.remove(mot)
        corpus_ensemble_documents[i] = " ".join(L)
    return (corpus_ensemble_documents)

In [4]:
from numpy import *

In [5]:
def TF(terme,corpus, numero_document):
    x=corpus[numero_document].count(terme)
    y=len(corpus[numero_document].split())
    return x/y

def IDF(terme, corpus, numero_document):
    D=len(corpus)
    d_t=0
    for document in corpus:
        if terme in document:
            d_t += 1
    TF_val = TF(terme, corpus, numero_document)
    return TF_val * log(1+(D/d_t))

def cles_correspondante_a_valeur(valeur, dictionnaire):
    for cle in dictionnaire.keys():
        if dictionnaire[cle]==valeur:
            return cle